# DataFrame object
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
import pyspark

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 43.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=93d43a020adbe41c5226561b8128459e568a9282f936c2b02d8a2a83b3d8b2a3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


## Create SparkContext and SparkSession

In [ ]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

## Create a DataFrame object

### Creat DataFrame by reading a file

In [ ]:
mtcars = spark.read.csv(path='/content/mtcars.csv',
                        sep=',',
                        encoding='UTF-8',
                        comment=None,
                        header=True, 
                        inferSchema=True)


mtcars.show(n=5, truncate=False)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|model            |mpg |cyl|disp |hp |drat|wt   |qsec |vs |am |gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|Mazda RX4        |21.0|6  |160.0|110|3.9 |2.62 |16.46|0  |1  |4   |4   |
|Mazda RX4 Wag    |21.0|6  |160.0|110|3.9 |2.875|17.02|0  |1  |4   |4   |
|Datsun 710       |22.8|4  |108.0|93 |3.85|2.32 |18.61|1  |1  |4   |1   |
|Hornet 4 Drive   |21.4|6  |258.0|110|3.08|3.215|19.44|1  |0  |3   |1   |
|Hornet Sportabout|18.7|8  |360.0|175|3.15|3.44 |17.02|0  |0  |3   |2   |
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



In [ ]:
type(mtcars)

pyspark.sql.dataframe.DataFrame

In [ ]:
mtcars.dtypes

[('model', 'string'),
 ('mpg', 'double'),
 ('cyl', 'int'),
 ('disp', 'double'),
 ('hp', 'int'),
 ('drat', 'double'),
 ('wt', 'double'),
 ('qsec', 'double'),
 ('vs', 'int'),
 ('am', 'int'),
 ('gear', 'int'),
 ('carb', 'int')]

In [ ]:
mtcars.describe().show()

+-------+-----------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|summary|      model|               mpg|               cyl|              disp|               hp|              drat|                wt|              qsec|                vs|                 am|              gear|              carb|
+-------+-----------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|  count|         32|                32|                32|                32|               32|                32|                32|                32|                32|                 32|                32|                32|
|   mean|       null|20.090624999999996|            6.1875|230.7218750000000

### Create DataFrame with `createDataFrame` function

#### From an RDD

Elements in RDD has to be an Row object

In [ ]:
from pyspark.sql import Row

rdd = sc.parallelize([
    Row(x=[1,2,3], y=['a','b','c']),
    Row(x=[4,5,6], y=['e','f','g'])
])

rdd.collect()

[Row(x=[1, 2, 3], y=['a', 'b', 'c']), Row(x=[4, 5, 6], y=['e', 'f', 'g'])]

In [ ]:
df = spark.createDataFrame(rdd)
df.show()

+---------+---------+
|        x|        y|
+---------+---------+
|[1, 2, 3]|[a, b, c]|
|[4, 5, 6]|[e, f, g]|
+---------+---------+



#### From pandas DataFrame

In [ ]:
import pandas as pd
pdf = pd.DataFrame({
    'x': [[1,2,3], [4,5,6]],
    'y': [['a','b','c'], ['e','f','g']]
})
pdf

,x,y
0,"[1, 2, 3]","[a, b, c]"
1,"[4, 5, 6]","[e, f, g]"


In [ ]:
df = spark.createDataFrame(pdf)
df.show()

+---------+---------+
|        x|        y|
+---------+---------+
|[1, 2, 3]|[a, b, c]|
|[4, 5, 6]|[e, f, g]|
+---------+---------+



In [ ]:
import pandas as pd
mtcars_pandas = pd.read_csv('/content/mtcars.csv')
print(mtcars_pandas.head())

mtcars_spark = spark.createDataFrame(mtcars_pandas)
print(mtcars_spark.head(5))

               model   mpg  cyl   disp   hp  ...   qsec  vs  am  gear  carb
0          Mazda RX4  21.0    6  160.0  110  ...  16.46   0   1     4     4
1      Mazda RX4 Wag  21.0    6  160.0  110  ...  17.02   0   1     4     4
2         Datsun 710  22.8    4  108.0   93  ...  18.61   1   1     4     1
3     Hornet 4 Drive  21.4    6  258.0  110  ...  19.44   1   0     3     1
4  Hornet Sportabout  18.7    8  360.0  175  ...  17.02   0   0     3     2

[5 rows x 12 columns]
[Row(model='Mazda RX4', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.62, qsec=16.46, vs=0, am=1, gear=4, carb=4), Row(model='Mazda RX4 Wag', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.875, qsec=17.02, vs=0, am=1, gear=4, carb=4), Row(model='Datsun 710', mpg=22.8, cyl=4, disp=108.0, hp=93, drat=3.85, wt=2.32, qsec=18.61, vs=1, am=1, gear=4, carb=1), Row(model='Hornet 4 Drive', mpg=21.4, cyl=6, disp=258.0, hp=110, drat=3.08, wt=3.215, qsec=19.44, vs=1, am=0, gear=3, carb=1), Row(model='Hornet Sportabou

In [ ]:
type(mtcars_spark)

pyspark.sql.dataframe.DataFrame

In [ ]:
mtcars_spark.corr('mpg','cyl')

-0.8521619594266131

In [ ]:
mtcars_spark.cov('mpg','cyl')

-9.172379032258062

In [ ]:
mtcars_spark.crosstab('mpg','gear').show()

+--------+---+---+---+
|mpg_gear|  3|  4|  5|
+--------+---+---+---+
|    18.1|  1|  0|  0|
|    15.8|  0|  0|  1|
|    30.4|  0|  1|  1|
|    27.3|  0|  1|  0|
|    15.2|  2|  0|  0|
|    33.9|  0|  1|  0|
|    14.3|  1|  0|  0|
|    17.8|  0|  1|  0|
|    24.4|  0|  1|  0|
|    16.4|  1|  0|  0|
|    22.8|  0|  2|  0|
|    21.0|  0|  2|  0|
|    21.5|  1|  0|  0|
|    21.4|  1|  1|  0|
|    32.4|  0|  1|  0|
|    19.7|  0|  0|  1|
|    17.3|  1|  0|  0|
|    18.7|  1|  0|  0|
|    10.4|  2|  0|  0|
|    15.5|  1|  0|  0|
+--------+---+---+---+
only showing top 20 rows



#### From a list

Each element in the list becomes an Row in the DataFrame. 

In [ ]:
alist = [['list',100,True]]
adf = spark.createDataFrame(alist,)
adf.show()
adf.dtypes

+----+---+----+
|  _1| _2|  _3|
+----+---+----+
|list|100|true|
+----+---+----+



[('_1', 'string'), ('_2', 'bigint'), ('_3', 'boolean')]

In [ ]:
my_list = [['a', 1], ['b', 2]]
df = spark.createDataFrame(my_list, ['letter', 'number'])
df.show()

+------+------+
|letter|number|
+------+------+
|     a|     1|
|     b|     2|
+------+------+



In [ ]:
df.dtypes

[('letter', 'string'), ('number', 'bigint')]

In [ ]:
my_list = [['a', 1], ['b', 2]]
df = spark.createDataFrame(my_list, ['my_column'])
df.show()

+---------+---+
|my_column| _2|
+---------+---+
|        a|  1|
|        b|  2|
+---------+---+



In [ ]:
df.dtypes

[('my_column', 'string'), ('_2', 'bigint')]

The following code generates a DataFrame consisting of two columns, each column is a vector column.

Why vector columns are generated in this case?
In this case, the list **my_list** has only one element, a tuple. Therefore, the DataFrame has only one row. This tuple has two elements. Therefore, it generates a two-columns DataFrame. Each element in the tuple is a list, so the resulting columns are vector columns.

In [ ]:
my_list = [(['a', 1], ['b', 2])]
df = spark.createDataFrame(my_list, ['x', 'y'])
df.show()

+------+------+
|     x|     y|
+------+------+
|[a, 1]|[b, 2]|
+------+------+





## Column instance

Column instances can be created in two ways:

1. directly select a column out of a *DataFrame*: `df.colName`
2. create from a column expression: `df.colName + 1`

Technically, there is only one way to create a column instance. Column expressions start from a column instance.

**Remember how to create column instances, because this is usually the starting point if we want to operate DataFrame columns.**

The column classes come with some methods that can operate on a column instance. ***However, almost all functions from the `pyspark.sql.functions` module take one or more column instances as argument(s)***. These functions are important for data manipulation tools.

## DataFrame column methods

### Methods that take column names as arguments:

* `corr(col1, col2)`: two column names.
* `cov(col1, col2)`: two column names.
* `crosstab(col1, col2)`: two column names.
* `describe(*cols)`: ***`*cols` refers to only column names (strings).***

### Methods that take column names or column expressions or **both** as arguments:

* `cube(*cols)`: column names (string) or column expressions or **both**.
* `drop(*cols)`: ***a list of column names OR a single column expression.***
* `groupBy(*cols)`: column name (string) or column expression or **both**.
* `rollup(*cols)`: column name (string) or column expression or **both**.
* `select(*cols)`: column name (string) or column expression or **both**.
* `sort(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sortWithinPartitions(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `orderBy(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sampleBy(col, fractions, sed=None)`: a column name.
* `toDF(*cols)`: **a list of column names (string).**
* `withColumn(colName, col)`: `colName` refers to column name; `col` refers to a column expression.
* `withColumnRenamed(existing, new)`: takes column names as arguments.
* `filter(condition)`: ***condition** refers to a column expression that returns `types.BooleanType` of values. 

ANOTHER METHOD

In [ ]:
!pip install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 23 kB/s 
     |████████████████████████████████| 198 kB 50.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=89af3f12e490e12029af3db773936a51bcd97964b24cc72accfa5a6de51cd6af
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('mtcars').getOrCreate()

In [ ]:
df = spark.read.csv('/content/mtcars.csv', 
                    header = True, 
                    inferSchema = True)

In [ ]:
df.show(n=5, truncate=False)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|_c0              |mpg |cyl|disp |hp |drat|wt   |qsec |vs |am |gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|Mazda RX4        |21.0|6  |160.0|110|3.9 |2.62 |16.46|0  |1  |4   |4   |
|Mazda RX4 Wag    |21.0|6  |160.0|110|3.9 |2.875|17.02|0  |1  |4   |4   |
|Datsun 710       |22.8|4  |108.0|93 |3.85|2.32 |18.61|1  |1  |4   |1   |
|Hornet 4 Drive   |21.4|6  |258.0|110|3.08|3.215|19.44|1  |0  |3   |1   |
|Hornet Sportabout|18.7|8  |360.0|175|3.15|3.44 |17.02|0  |0  |3   |2   |
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
df.dtypes

[('_c0', 'string'),
 ('mpg', 'double'),
 ('cyl', 'int'),
 ('disp', 'double'),
 ('hp', 'int'),
 ('drat', 'double'),
 ('wt', 'double'),
 ('qsec', 'double'),
 ('vs', 'int'),
 ('am', 'int'),
 ('gear', 'int'),
 ('carb', 'int')]

In [ ]:
df.describe().show()

+-------+-----------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|summary|        _c0|               mpg|               cyl|              disp|               hp|              drat|                wt|              qsec|                vs|                 am|              gear|              carb|
+-------+-----------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|  count|         32|                32|                32|                32|               32|                32|                32|                32|                32|                 32|                32|                32|
|   mean|       null|20.090624999999996|            6.1875|230.7218750000000

In [ ]:
from pyspark.sql import Row

rdd = sc.parallelize([
    Row(x=[1,2,3], y=['a','b','c']),
    Row(x=[4,5,6], y=['e','f','g'])
])

rdd.collect()

In [ ]:
df.corr('mpg','cyl')

-0.8521619594266132

In [ ]:
df.cov('mpg','cyl')

-9.172379032258062

In [ ]:
df.crosstab('mpg','gear').show()

+--------+---+---+---+
|mpg_gear|  3|  4|  5|
+--------+---+---+---+
|    18.1|  1|  0|  0|
|    15.8|  0|  0|  1|
|    30.4|  0|  1|  1|
|    27.3|  0|  1|  0|
|    15.2|  2|  0|  0|
|    33.9|  0|  1|  0|
|    14.3|  1|  0|  0|
|    17.8|  0|  1|  0|
|    24.4|  0|  1|  0|
|    16.4|  1|  0|  0|
|    22.8|  0|  2|  0|
|    21.0|  0|  2|  0|
|    21.5|  1|  0|  0|
|    21.4|  1|  1|  0|
|    32.4|  0|  1|  0|
|    19.7|  0|  0|  1|
|    17.3|  1|  0|  0|
|    18.7|  1|  0|  0|
|    10.4|  2|  0|  0|
|    15.5|  1|  0|  0|
+--------+---+---+---+
only showing top 20 rows

